In [5]:
import numpy as np
import json
from pycldf.dataset import Dataset
import pandas as pd
from numpy.linalg import norm
from scipy import spatial
from zipfile import ZipFile as zf
import gram2vec.lang2vec.lang2vec as l2v
import pickle
from helpers import to_iso

In [7]:
# Reads in gold standard MT scores for languages and finds the ranking
import scipy.stats as ss
import pandas as pd
POS_XLMR = "./golds/xpos_scores.csv"

data = pd.read_csv(POS_XLMR)


In [8]:
pos = pickle.load(open("training-data/POS_gram_golds_no_ties.pkl", 'rb'))

In [33]:
!pip install langcodes

DEPRECATION: Loading egg at /Users/CitronVert/opt/anaconda3/envs/lang2vec-clean/lib/python3.12/site-packages/lang2vec-1.1.6-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)


In [9]:
import pandas as pd
import pickle
from langcodes import Language

def to_iso(lang):
    return Language.make(lang).to_alpha3()

pos = pickle.load(open("training-data/POS_gram_golds_no_ties.pkl", 'rb'))
gold_langs = set(pos.keys())
POS_XLMR = "./golds/xpos_scores.csv"
data = pd.read_csv(POS_XLMR)

data.columns = [to_iso(col) if col != "lang_train" else col for col in data.columns]# Convert contents of the first column from 2-letter to 3-letter ISO codes

data['lang_train'] = data['lang_train'].apply(lambda x: to_iso(x) if pd.notnull(x) else x)
filtered_data = data[data['lang_train'].isin(gold_langs)]

# Filter out columns with language codes not present in gold_langs
filtered_data = filtered_data[filtered_data.columns[0:1].tolist() + [col for col in filtered_data.columns[1:] if col in gold_langs]]

In [10]:
filtered_data = filtered_data[filtered_data['lang_train'].isin(filtered_data.columns)]

In [11]:
languages = list(filtered_data['lang_train'].unique()) + ["lang_train"]
filtered_columns = [col for col in filtered_data.columns if col in languages]
filtered_data = filtered_data[filtered_columns]

In [31]:
filtered_data

,lang_train,bel,gle,hun,hye,lit,mar,tam
2,bel,98.282538,59.220108,76.205972,87.359349,89.862254,85.276074,85.448916
20,gle,79.140645,86.046512,77.143951,71.750060,76.381656,79.141104,78.095975
28,hun,77.310767,59.830867,97.339204,85.683505,79.674114,87.116564,82.739938
29,hye,91.918561,62.426591,85.729326,97.031362,89.005543,83.435583,86.919505
37,lit,89.145641,60.104925,76.885528,84.055542,96.128003,82.822086,79.140867
40,mar,65.400949,50.140944,69.410413,74.431410,75.919704,88.957055,73.529412
56,tam,76.942293,58.237413,69.783691,76.466363,77.767512,79.141104,85.642415


In [12]:
def make_gold_xpos(data):
    indices = data.columns 
    indices = list(indices[1:]) # creates final list of languages
    langs_ranked_ties = {}
    # creates a ranking for each language (with the language itself removed from consideration)
    for language in indices: 
        values = list(data[language])
        values = [-float(a) for a in values]
        ranked =  ss.rankdata(values, method = "ordinal") #ranks remaining languages (ties are assigned the minimum ranking that would be assigned if no ties were presen)
        langs_ranked_ties[language] = (indices, ranked)
    return langs_ranked_ties

def make_xpos_rank(data, lang):
    remove = [lang]
    data = data.drop(remove,axis = 1) #columns that match undesirable language codes
    data = data.drop(data.loc[data["lang_train"].isin(remove)].index) # removes rows that match undesirable language codes
    indices = data.columns 
    indices = list(indices[1:]) # creates final list of languages
    rankings = {}
    for lang in indices:
        values = list(data[lang])
        values = [-float(a) for a in values]
        ranked =  ss.rankdata(values, method = "ordinal") #ranks remaining languages (ties are assigned the minimum ranking that would be assigned if no ties were presen)
        ranked = [i - 1 for i in ranked]
        rankings[lang] = ranked
    return (indices, rankings)

In [13]:
gram_golds = make_gold_xpos(filtered_data)
with open("./training-data/POS_XLMR_gram_golds_no_ties.pkl", 'wb') as f:
    pickle.dump(gram_golds, f)

training = {}
for language in filtered_data.columns[1:]:
    ranking = make_xpos_rank(filtered_data, language)
    training[language] = ranking
with open("./training-data/POS_XLMR_gram_ranked_train_no_ties.pkl", 'wb') as f:
    pickle.dump(training, f)


In [33]:
training

{'bel': (['gle', 'hun', 'hye', 'lit', 'mar', 'tam'],
  {'gle': [0, 3, 1, 2, 5, 4],
   'hun': [2, 0, 1, 3, 5, 4],
   'hye': [5, 1, 0, 2, 4, 3],
   'lit': [4, 2, 1, 0, 5, 3],
   'mar': [4, 1, 2, 3, 0, 5],
   'tam': [4, 2, 0, 3, 5, 1]}),
 'gle': (['bel', 'hun', 'hye', 'lit', 'mar', 'tam'],
  {'bel': [0, 3, 1, 2, 5, 4],
   'hun': [3, 0, 1, 2, 5, 4],
   'hye': [1, 2, 0, 3, 5, 4],
   'lit': [1, 3, 2, 0, 5, 4],
   'mar': [2, 1, 3, 4, 0, 5],
   'tam': [2, 3, 0, 4, 5, 1]}),
 'hun': (['bel', 'gle', 'hye', 'lit', 'mar', 'tam'],
  {'bel': [0, 3, 1, 2, 5, 4],
   'gle': [3, 0, 1, 2, 5, 4],
   'hye': [1, 5, 0, 2, 4, 3],
   'lit': [1, 4, 2, 0, 5, 3],
   'mar': [1, 4, 2, 3, 0, 5],
   'tam': [2, 4, 0, 3, 5, 1]}),
 'hye': (['bel', 'gle', 'hun', 'lit', 'mar', 'tam'],
  {'bel': [0, 2, 3, 1, 5, 4],
   'gle': [3, 0, 2, 1, 5, 4],
   'hun': [3, 1, 0, 2, 5, 4],
   'lit': [1, 4, 2, 0, 5, 3],
   'mar': [2, 4, 1, 3, 0, 5],
   'tam': [1, 4, 2, 3, 5, 0]}),
 'lit': (['bel', 'gle', 'hun', 'hye', 'mar', 'tam'],
  {'bel

In [68]:
numeric_df = data.drop(columns=['lang_train']).apply(pd.to_numeric)


In [87]:
df = data.copy()

for column in df.columns:
    # Set entries to null where the value in 'lang_train' matches the column name
    df.loc[df['lang_train'] == column, column] = np.nan

numeric_df = df.drop(columns=['lang_train']).apply(pd.to_numeric)


ranges = numeric_df.max(skipna=True) - numeric_df.min(skipna=True)# Find the column with the maximum range
column_with_max_span = ranges.idxmax()

print("Column with the greatest span of values:", column_with_max_span)
min_val = numeric_df[column_with_max_span].min()
max_val = numeric_df[column_with_max_span].max()
min_index = data.loc[data[column_with_max_span] == min_val, "lang_train"].iloc[0]
max_index = data.loc[data[column_with_max_span] == max_val, "lang_train"].iloc[0]
print(f"{column_with_max_span}: Min Value: {min_val} (lang: {min_index}), Max Value: {max_val} (lang: {max_index})")

Column with the greatest span of values: cat
cat: Min Value: 25.72064229057845 (Index: san), Max Value: 95.65196362932868 (Index: spa)


In [ ]:
# data = data[:54] #cuts off the top 3 data from the bottom of the csv (MT)
# data = data[:31] # (DEP)
# data = data[:60] # (POS)
# data = data[:54] #(EL)


def generate_rankings(data, remove):
    remove = list(set(remove) & set(data.columns))
    data = data.drop(remove, axis = 1) # drop test languages (columns) that match undesirable language codes
    data = data.drop(data.loc[data['v extra v \ trg >>'].isin(remove)].index) # removes rows that match undesirable language codes
    indices = data.columns 
    indices = list(indices[1:]) # creates final list of languages
    langs_ranked_ties = {}
    # creates a ranking for each language (with the language itself removed from consideration)
    for language in indices: 
        values = list(data[language])
        values = [-float(a) for a in values]
        ranked =  ss.rankdata(values, method = "ordinal") #ranks remaining languages (ties are assigned the minimum ranking that would be assigned if no ties were presen)
        langs_ranked_ties[language] = (indices, ranked)
    return langs_ranked_ties


def generate_ranking(data, language, r):
    # remove.append(language)
    remove = r.copy()
    remove.append(language)
    remove = list(set(remove) & set(data.columns))
    data = data.drop(remove,axis = 1) #columns that match undesirable language codes
    data = data.drop(data.loc[data['v extra v \ trg >>'].isin(remove)].index) # removes rows that match undesirable language codes
    indices = data.columns 
    indices = list(indices[1:]) # creates final list of languages
    langs_ranked_ties = {}
    # creates a ranking for each language (with the language itself removed from consideration)
    rankings = {}
    for lang in indices:
        values = list(data[lang])
        values = [-float(a) for a in values]
        ranked =  ss.rankdata(values, method = "ordinal") #ranks remaining languages (ties are assigned the minimum ranking that would be assigned if no ties were presen)
        ranked = [i - 1 for i in ranked]
        rankings[lang] = ranked
    return (indices, rankings)

In [ ]:
data = data.rename(columns={'v pivot v \ test >>':'v extra v \ trg >>'}) #stupid POS/EL inconsistency 
# data = data.rename(columns={'Unnamed: 0':'v extra v \ trg >>'}) #stupid POS/EL inconsistency 
    


In [ ]:
# finds the intersection of the grambank languages and task languages
# finds the rankings for each language with non-intersection languages removed and saves to a pickle file

import pickle
import gram2vec.lang2vec.lang2vec as l2v

gram = l2v.available_gram()


col_langs = to_iso(list(data.columns)[1:])
print("there  were {l} test languages originally. Those languages were {col}.".format(l = len(col_langs), col = col_langs))

lang = list(data['v extra v \ trg >>'])
lang = to_iso(lang)
missing = [a for a in col_langs if not a in lang]
print("there were  {l} languages excluded for not being present in ranking data. Those languages were {missing}.".format(l = len(missing), missing = missing))


data['v extra v \ trg >>'] = lang #reassigns ranking langs to ISO-3 codes
data.columns = ['v extra v \ trg >>'] + col_langs #reassigns test languages to ISO-3 codes
data = data.drop(missing,axis = 1) #drop test languages (columns) that aren't present in ranking languages (for now)

col_langs = list(data.columns)[1:]  #reassigns column headers to remaining test languages
print("After dropping missing languages, there are {l} test languages remaining. Those languages were {col}.".format(l = len(col_langs), col = col_langs))


common_test = (set(gram) & set(col_langs)) # final set of test languages (in grambank AND in ranking languages)

total_langs = lang + col_langs

common= (set(gram) & set(total_langs))
remove = list(set(total_langs) ^ common) # languages that should be removed (not in grambank)
print("there were {l} total languages excluded for not being present in grambank. Those languages were {missing}.".format(l = len(remove), missing = remove))

training = {}
gram_golds = generate_rankings(data, remove)
with open("./training-data/MT_gram_golds_no_ties.pkl", 'wb') as f:
    pickle.dump(gram_golds, f)

for language in common_test:
    ranking = generate_ranking(data, language, remove)
    training[language] = ranking
with open("./training-data/MT_ranked_train_no_ties.pkl", 'wb') as f:
    pickle.dump(training, f)



In [2]:
#Loads in list of languages that have precomputed distances in lang2vec
with open("./gram2vec/lang2vec/data/distances/distances_languages.txt") as f:
    dist_langs = f.read()
dist_langs =  dist_langs.split(",")

In [3]:
# Extract ISO6393 codes from... somewhere
with open('/Users/CitronVert/Downloads/resourcemap.json') as f:
  
# returns JSON object as 
# a dictionary
    data = json.load(f)

def get_iso6393(identifiers):
    for identifier in identifiers:
        if identifier.get("type") == "iso639-3":
            return identifier.get("identifier")
    return None

# Use list comprehension to extract the required fields
glotto2iso = {
        resource.get("id"): get_iso6393(resource.get("identifiers", []))
        for resource in data.get("resources", [])
        if get_iso6393(resource.get("identifiers", [])) is not None
    }

iso2glotto = {v: k for k, v in glotto2iso.items()}


In [4]:
# replace language codes in the vector data with ISO codes

from pycldf.dataset import Dataset
# metadata = "/Users/CitronVert/Desktop/grambank-grambank-analysed-fcf971a/grambank/cldf/StructureDataset-metadata.json"
imputed = "/Users/CitronVert/Desktop/NALA/grambank-grambank-analysed-fcf971a/R_grambank/output/GB_wide/GB_wide_imputed_binarized.tsv"
v = pd.read_csv(imputed, sep='\t')
for lang in v["Language_ID"]:
    if lang in glotto2iso.keys():
        v.loc[ v["Language_ID"] == lang, "Language_ID"] = glotto2iso[lang]
    else:
        v.drop(v[v['Language_ID'] == lang].index, inplace = True)


In [5]:
feats = list(v.columns[1:])


In [6]:
categories = "/Users/CitronVert/Downloads/feature_grouping_with_alternatives.csv"
gram_categories = pd.read_csv(categories)
cats = {}
groups = {}
for feat in feats:
    if not len(feat) == 5:
        feat = feat[:5]
    domain = gram_categories.loc[gram_categories["Feature_ID"] == feat, 'Main_domain'].item()
    fine = gram_categories.loc[gram_categories["Feature_ID"] == feat, 'Finer_grouping'].item()
    if fine in groups.keys():
        groups[fine].append(feat)
    else:
        groups[fine] = [feat]
    if domain in cats.keys():
        cats[domain].append(feat)
    else:
        cats[domain] = [feat]


In [7]:
#data: pandas array with grambank vector data (imputed and converted to iso codes)
#iso: string ISO code for a language
#reuturns the grambank feature vector
def get_vector(data, iso, sub_feats=None):
    uriel = np.array(l2v.get_features(iso, "syntax_knn", sub_feats=sub_feats)[iso])
    # gram = np.array(data[data['Language_ID'] == iso].values.tolist()[0][1:])
    # concat = np.concatenate((uriel, gram))
    return uriel

In [8]:
#finds the languages that are present in both lang2vec and grambank and reports the length
with open("/Users/CitronVert/Desktop/NALA/bankrank/gram2vec/lang2vec/data/distances/intersection_langs.txt", 'r') as file:
    intersection = file.read()
intersection = intersection.split(",")
d =len(intersection)

In [9]:
#read in csv
fine_cat ="/Users/CitronVert/Desktop/NALA/bankrank data/fine_categorization.csv"
cats = pd.read_csv(fine_cat)

In [10]:
np_order = cats["Uriel"][12].split(",")
clause = cats["Uriel"][11].split(",")


In [11]:
np_order=[a.strip() for a in np_order]
clause = [a.strip() for a in clause]

print(len(np_order))
print(len(clause))


16
41


In [12]:
feat_vecs = [get_vector(v, iso) for iso in intersection]
sub_np_feat_vecs = [get_vector(v, iso, np_order) for iso in intersection]
sub_clause_feat_vecs = [get_vector(v, iso, clause) for iso in intersection]

In [13]:
sub_np_feat_vecs[1]

array([0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
       0., 0.])

In [14]:
feat_vecs[1]

array([0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0.,
       1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
       0.])

In [15]:
feat_arr = np.stack(feat_vecs)
sub_np_feat_arr =  np.stack(sub_np_feat_vecs)
sub_clause_feat_arr =  np.stack(sub_clause_feat_vecs)

# print(sub_np_feat_arr.shape)
# print(sub_clause_feat_arr.shape)

In [18]:
import sklearn.metrics.pairwise as sk
import scipy.sparse as sparse

cosine_similarities = sk.cosine_similarity(feat_arr)
knn = sparse.csr_matrix(1-cosine_similarities)
sparse.save_npz("./URIEL_knn_distances.npz", knn)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
import scipy.sparse as sparse

np = sparse.load_npz("/Users/CitronVert/Desktop/NALA/bankrank/gram2vec/lang2vec/data/distances/URIEL_knn_distances_np_order_ablation.npz")
clause = sparse.load_npz("/Users/CitronVert/Desktop/NALA/bankrank/gram2vec/lang2vec/data/distances/URIEL_knn_distances_clause_ablation.npz")
reg = sparse.load_npz("/Users/CitronVert/Desktop/NALA/bankrank/gram2vec/lang2vec/data/distances/URIEL_knn_distances.npz")


In [21]:
print(reg)


  (0, 0)	-2.220446049250313e-16
  (0, 1)	0.5890025317366068
  (0, 2)	0.05370837673721851
  (0, 3)	0.4856555001263603
  (0, 4)	0.3043344070000654
  (0, 5)	0.48550424457247354
  (0, 6)	0.3617152614957745
  (0, 7)	0.4645503299035949
  (0, 8)	0.6284197321912309
  (0, 9)	0.41666666666666663
  (0, 10)	0.5068030380839282
  (0, 11)	0.3036893761772086
  (0, 12)	0.06840573860297583
  (0, 13)	0.34927599213080796
  (0, 14)	0.5628071751074061
  (0, 15)	0.452003375648809
  (0, 16)	0.571253537143728
  (0, 17)	0.4246035444312495
  (0, 18)	0.6712020253892854
  (0, 19)	0.2602045571258924
  (0, 20)	0.04100590738541632
  (0, 21)	0.38888888888888873
  (0, 22)	0.6055946811266921
  (0, 23)	0.09850212828958171
  (0, 24)	0.5463035642189196
  :	:
  (1469, 1445)	0.4997835966139753
  (1469, 1446)	0.5454545454545454
  (1469, 1447)	0.4848725736710717
  (1469, 1448)	0.4276361929678575
  (1469, 1449)	0.4197411468143405
  (1469, 1450)	0.4997835966139753
  (1469, 1451)	0.5067799747921895
  (1469, 1452)	0.28454524120982

In [ ]:
GRAMBANK_LANGUAGES = l2v.available_grambank()
GRAMBANK_DISTANCES =  "/Users/CitronVert/Desktop/langrank/gram2vec/lang2vec/data/learned.npy"
with open(GRAMBANK_DISTANCES, "rb") as f:
    feat_pred = np.load(GRAMBANK_DISTANCES, allow_pickle=True)

In [ ]:
feat_pred

In [ ]:
#for every pair of languages that is present in both grambank and lang2vec,
#find the cosine difference between their feature vectors
import scipy.sparse as sparse

res = np.zeros((d,d))
for x in range(d):
    l1 = intersection[x]
    v1 = get_vector(v, l1)
    for y in range(x,d):
        l2 = intersection[y]
        v2 = get_vector(v, l2)
        diff = spatial.distance.cosine(v1, v2)
        res[x][y] = diff


# #saves the distance measures as a sparse matrix (for compatability with lang2vec)
sA = sparse.csr_matrix(res)
sparse.save_npz("./concat_distances.npz", sA)
               

In [ ]:
intersection

In [ ]:
x = intersection.index("mij")
y = intersection.index("sed")

In [ ]:
res[y][x]

In [ ]:
#saves a list of the languages for which we have distance values
data_string = ",".join(str(item) for item in intersection)

    # Open the file in write mode
with open("./URIEL_distance_langs.txt", 'w') as file:
    # Write the data string to the file
    file.write(data_string)

In [ ]:
# training code for reference
# def test_train():
#     langs = ["aze", "ben", "fin"]
#     datasets = [os.path.join(root, "sample-data", "ted-train.orig.{}".format(l)) for l in langs]
#     seg_datasets = [os.path.join(root, "sample-data", "ted-train.orig.spm8000.{}".format(l)) for l in langs]
#     rank = [[0, 1, 2], [1, 0, 2], [2, 1, 0]] # random
#     tmp_dir = "tmp"
#     prepare_train_file(datasets=datasets, segmented_datasets=seg_datasets,
#                        langs=langs, rank=rank, tmp_dir=tmp_dir, task="MT")
#     output_model = "{}/model.txt".format(tmp_dir)
#     train(tmp_dir=tmp_dir, output_model=output_model)
#     assert os.path.isfile(output_model)

In [ ]:
TED_PATH = "/Users/CitronVert/Desktop/langrank/indexed/MT/ted.npy"
def create_MT_dataset(data_path, lang): 
    data = np.load(data_path, encoding='latin1', allow_pickle=True).item()
    filename = "ted-train.orig."+lang
    


def train(langs, datasets, seg_datasets, rank): 
    tmp_dir = "tmp"
    prepare_train_file(datasets=datasets, segmented_datasets=seg_datasets,
                       langs=langs, rank=rank, tmp_dir=tmp_dir, task="MT")
    output_model = "{}/model.txt".format(tmp_dir)
    train(tmp_dir=tmp_dir, output_model=output_model)
    assert os.path.isfile(output_model)

In [ ]:
#function to compute the ndcg score
#lang: ISO code for task language
#ranked_langs: gold rankings for task
#predicted: predicted ranking for lang
#gamma_max: hyper param from lin et al.

import sklearn.metrics as sm
import numpy as np
def compute_ndcg(lang, ranked_langs, predicted, gamma_max= 10):
    ranking_langs = ranked_langs[lang][0] # list of languages for looking up index in ranking vector
    # gives position in ranking based on index (if ranking[0] = 4 then the 0th language [ranking_langs[0]] is the 4th best)
    ranking = ranked_langs[lang][1] 
    # creates vector to look up the relevance score of a given language by index
    scores_by_index = [0] * len(ranking)
    for i in range(len(ranking)): 
        if ranking[i] <= gamma_max:
            scores_by_index[i] = gamma_max - (ranking[i] - 1)
    print("scores by index")
    print(scores_by_index)
    ideal_score = [i for i in reversed(range(1, gamma_max + 1))] + [0] * (len(ranking) - gamma_max)
    print("IDEAL")
    print(ideal_score)
    predicted_score = [0] * len(ranking)
    for j in range(len(predicted)): #for each language in ranking
        code = predicted[j]
        print(code)
        index = ranking_langs.index(code)
        print(ranking[index])
        score = scores_by_index[index] #finds the true relevance of each language
        predicted_score[j] = score
    return sm.ndcg_score(np.asarray([ideal_score]), np.asarray([predicted_score]))

In [ ]:
# a small ndcg test
# remember-- we don't exclude the test language
dummy = {
    "aze": (["ben", "fin", "deu", "eng"],[1, 2, 3, 4]),
    "ben": (["aze","fin", "deu", "eng"],[1, 3, 2, 4]),
    "fin": (["aze", "ben", "deu", "eng"],[3, 4, 2, 1]),
    "deu": (["aze", "ben", "fin", "eng"],[4, 3, 2, 1]),
    "eng": (["aze", "ben", "fin", "deu"],[4, 3, 2, 1])
}
lang = "eng"
predicted = ["fin","deu","aze","ben"]
gamma_max = 2 
print(compute_ndcg(lang, dummy, predicted, gamma_max))

In [ ]:
# Modify the code to accept params instead of freakin data
# pull list of MT languages from ... somewhere
# run write a function to run langrank_predict with the all model for all langs in list and return ndcg
# average list
# is it same??

In [ ]:
import langrank as lr
import pickle
with open("./MT_ranked_ties.pkl", 'rb') as f:
    rankings = pickle.load(f)
languages = list(rankings.keys())

predicted = {}
for lang in languages:
    prepared = lr.prepare_featureset(lang)
    predicted[lang] = lr.rank(prepared, task="MT", candidates="all", return_langs = True, uriel = True)




In [ ]:
with open("./MT_full_ranked_no_ties.pkl", 'rb') as f:
    rankings = pickle.load(f)
languages = list(rankings.keys())
rank = rankings["aze"][1]
gold = [0]*len(languages)
for i in range(len(languages)):
    gold[rank[i]-1] = languages[i]
print(gold)

In [ ]:
import pickle
with open("./uriel_predictions.pkl", 'rb') as f:
    predictions = pickle.load(f)
print(predictions["aze"])

In [ ]:
with open("./MT_full_ranked_ties.pkl", 'rb') as f:
    rankings = pickle.load(f)

In [ ]:
lang = "aze"
ranking_langs = rankings[lang][0] # list of languages for looking up index in ranking vector
ranking = rankings[lang][1] 


In [ ]:
langs = [""] * len(ranking_langs)
for i in range(len(ranking_langs)):
    langs[ranking[i] - 1] = ranking_langs[i]

In [ ]:
langs

In [ ]:
import numpy as np

In [ ]:
tasks = {"MT", "EL", "DEP", "POS"}
cases = {"uriel", "gram"}
path = "/Users/CitronVert/Desktop/langrank/results"
for task in tasks:
    for case in cases:
        p = "{base}/{task}/{task}_{case}_predictions.pkl".format(base = path, task = task, case = case)
        with open(p, 'rb') as f:
            predictions = pickle.load(f)
        n = "{base}/{task}/{task}_{case}_ndcg.pkl".format(base = path, task = task, case = case)
        with open(n, 'rb') as f:
            scores = pickle.load(f)
        vals = [i for i in list(scores.values())]
        # vals
        predict = pd.DataFrame.from_dict(predictions)
        scores_d = pd.DataFrame( columns = list(scores.keys()))
        scores_d.loc[len(scores_d)]= vals 
        predict.to_csv(r"{base}/{task}/{task}_{case}_predictions.csv".format(base = path, task = task, case = case))
        scores_d.to_csv(r"{base}/{task}/{task}_{case}_ndcg.csv".format(base = path, task = task, case = case))



In [ ]:
scores_d.loc[len(scores_d)]= vals 

In [ ]:
import math
def percent_missing(lang, path):
    v = pd.read_csv(path, sep='\t')
    glotto = iso2glotto[lang]
    vector = list(get_vector(v, glotto))
    a = sum(math.isnan(x) for x in vector)
    return a/len(vector)

    

In [ ]:
def get_task_langs(task):
    with open("./training-data/{}_ranked_train_no_ties.pkl".format(task), 'rb') as f:
        rankings = pickle.load(f)
    languages = list(rankings.keys())
    return languages

In [ ]:
PATH = "/Users/CitronVert/Desktop/grambank-grambank-analysed-fcf971a/R_grambank/output/GB_wide/GB_cropped_for_missing.tsv"
tasks = ["MT", "DEP", "POS", "EL"]
for task in tasks:
    d = {}
    langs = get_task_langs(task)
    for lang in langs: 
        missing = percent_missing(lang, PATH)
        d[lang] = missing
    with open(r"./{task}_percent_missing.txt".format(task = task), 'w') as fp:
        for item in d.keys():
            percent = d[item]
            # write each item on a new line
            fp.write("{lang}: {percent}\n".format(lang = item, percent = percent))
        print('Done')


In [ ]:
with open(r'./uriel_features.txt', 'w') as fp:
    for item in features:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')